In [1]:
# 将原始图片资源里的人脸抽取做训练素材

In [2]:
#coding: UTF-8

# 包含库 ######################################################
# 绘制图片需要
from matplotlib import pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm

import os
import sys

# 引入功能库
import extractFace as feExtracter

In [3]:
# 获取文件名(带后缀)
def getFileName(fileUrl):
    filepath, tmpfilename = os.path.split(fileUrl)
    return tmpfilename
# print(getFileName("/home/wilson/aaa.png"))

# 将脸部存储到指定目录
def saveFace(faceInfo, faceImageSavePath):
    # 原图信息
    imagePath = bytes.decode(faceInfo.imagePath)
    image = cv2.imread(imagePath)
    sp = image.shape            #获取图像形状：返回【行数值，列数值】列表
    height = sp[0]                 #图像的高度（行 范围）
    width = sp[1]                 #图像的宽度（列 范围）

    # 修正截取位置
    top = max(0, faceInfo.top)
    bottom = min(height, faceInfo.bottom)
    left = max(0, faceInfo.left)
    right = min(width, faceInfo.right)
    cropImg = image[top:bottom, left:right] #裁剪图像
    #plt.imshow(cropImg)
    #plt.show()
    
    # 检验解码目录
    if not os.path.exists(faceImageSavePath):                                                                                                                    
         os.makedirs(faceImageSavePath)
    
    imageSavePath = faceImageSavePath+getFileName(imagePath)
    print(imageSavePath)
    cv2.imwrite(imageSavePath,cropImg)  #写入图像路径
    
# 抽取人脸
def extractFace(instance, nodeName, dirPath, faceImageSavePath):
    response = feExtracter.extractFace(instance, dirPath)
    if response.count < 1:
        print("no any faces extracted")
    else:
        print(nodeName, response.count, "face extracted")
        for k in tqdm(range(response.count)):
            e = response.elems[k]
            saveFace(e, faceImageSavePath + nodeName + "/")
        feExtracter.releaseFaceExtractResponse(instance, response)

In [5]:
# 配置原始资源目录，和人脸存放目录
rawImagePath = '/home/wilson/workstation/TFResFaceEmotion/raw/'
faceImageSavePath = '/home/wilson/workstation/TFResFaceEmotion/extract/'

# 创建实例
instance = feExtracter.createInstance()

# 遍历原始图片的根目录
for nodeName in os.listdir(rawImagePath):
    nodePath = rawImagePath + nodeName
    if os.path.isdir(nodePath): # 这是表情目录
        extractFace(instance, nodeName, nodePath, faceImageSavePath)

feExtracter.releaseInstance(instance)
print("Done")

100%|██████████| 23/23 [00:00<00:00, 641.81it/s]

sad 23 face extracted
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (7).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (9).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (23).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (16).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (28).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (4).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (22).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (24).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (26).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (21).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (25).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (8).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (12).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/sad/sad (20).jpg
/home/wilson/workstation/TFResFaceEmotion/ex


100%|██████████| 21/21 [00:00<00:00, 708.16it/s]

happy 21 face extracted
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (7).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (14).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (19).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (1).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (5).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (26).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (20).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (2).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (25).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (30).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (21).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (11).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/happy (22).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/happy/h


100%|██████████| 11/11 [00:00<00:00, 670.50it/s]

fear 11 face extracted
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (6).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (14).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (7).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (13).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (2).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (3).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (4).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (1).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (10).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (11).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/fear/fear (12).jpg



100%|██████████| 21/21 [00:00<00:00, 638.56it/s]

anger 21 face extracted
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (21).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (8).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (26).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (19).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (20).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (7).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (16).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (25).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (10).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (22).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (6).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (24).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/anger (17).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/anger/


100%|██████████| 3/3 [00:00<00:00, 726.87it/s]

disgust 3 face extracted
/home/wilson/workstation/TFResFaceEmotion/extract/disgust/disgust (5).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/disgust/disgust (1).jpg
/home/wilson/workstation/TFResFaceEmotion/extract/disgust/disgust (2).jpg
Done
